# Composite System Evolution
Here we try to model the evolution one system at a time, saving the intermediate state of ancillas

In [1]:
include("./src/RoutineFunctions.jl")
include("./modules/Thermodynamics.jl")
include("./modules/OpticalCavity.jl")
include("./modules/Phaseonium.jl")
include("./modules/BosonicOperators.jl")
include("./modules/Measurements.jl")

using LinearAlgebra
using ProgressBars
using Plots
using LaTeXStrings

using .Thermodynamics
using .OpticalCavity
using .Phaseonium
using .BosonicOperators
using .Measurements

In [3]:
NDIMS = 40
Ω = 1.0
Δt = 1e-2

T_initial = 1.0
T_heating = 1.5
T_cooling = 0.25

# Create a Cavity
α0 = π
l0 = 1.0
expanding_force = 0.5
compressing_force = 2.5
cavity = Cavity(1.0, 1.0, l0, α0, expanding_force)
ω = α0 / l0

ρt = thermalstate(NDIMS, ω, T_initial)
println(
    "Initial Temperature of the Cavity:
    $(Measurements.temperature(ρt, ω))")

# Jump Operators
a = BosonicOperators.destroy(NDIMS)
ad = BosonicOperators.create(NDIMS)

# Create Phaseonium atoms
# For thermal (diagonal) phaseoniums, the Master Equation is the same as having ϕ=π/2
# Heating
ϕ_h = π/3
α_h = Phaseonium.alpha_from_temperature(T_heating, ϕ_h) 

ga_h, gb_h = Phaseonium.dissipationrates(α_h, ϕ_h)
println(
    "Apparent Temperature carried by Hot Phaseonium atoms: 
    $(Phaseonium.finaltemperature(ω, ga_h, gb_h))")

# Cooling
ϕ_c = π/4
α_c = Phaseonium.alpha_from_temperature(T_cooling, ϕ_c) 

ga_c, gb_c = Phaseonium.dissipationrates(α_c, ϕ_c)
println(
    "Apparent Temperature carried by Cold Phaseonium atoms: 
    $(Phaseonium.finaltemperature(ω, ga_c, gb_c))")

# Time Evolution loop
const TT = 20000  # Thermalization Time

system_evolution = []
;

Initial Temperature of the Cavity:
    1.0
Apparent Temperature carried by Hot Phaseonium atoms: 
    4.71238898038469
Apparent Temperature carried by Cold Phaseonium atoms: 
    0.7853981633974483


### One-step evolution
We calculare:
\begin{align}
&\rho_1' = Tr_{\eta}\left\{ e^{-i\Delta t(H_S + H_\eta + V_n)}\rho_1\otimes\eta\, e^{i\Delta t(H_S + H_\eta + V_n)} \right\} \\
&\eta ' = Tr_{\rho_1}\left\{ e^{-i\Delta t(H_S + H_\eta + V_n)}\rho_1\otimes\eta\, e^{i\Delta t(H_S + H_\eta + V_n)} \right\} \\
&\rho_2' = Tr_{\eta'}\left\{ e^{-i\Delta t(H_S + H_\eta + V_n)}\rho_2\otimes\eta'\, e^{i\Delta t(H_S + H_\eta + V_n)} \right\}
\end{align}

with:

\begin{align}
&H_S = \hbar\omega\left(\hat{n}+\frac{1}{2}\right) \\
&H_\eta = \frac{\hbar\omega}{2} \left(\sigma^+_1\sigma^-_1 + \sigma^+_2\sigma^-_2\right) = \frac{\hbar\omega}{2}\left(\sigma_{z1}+\sigma_{z2}\right) \\
&V_n = \hbar\Omega \{ \hat{a}(\sigma_1^+ + \sigma_2^+) + \hat{a}^\dagger(\sigma_1^- + \sigma_2^-) \} =
     \hbar\Omega \{ \hat{a}\sigma^+ + \hat{a}^\dagger\sigma^- \}
\end{align}

In [5]:
function H_s(α, l, n)
    α / l * ( n + 1/2)
end

function H_eta(α, l, sz)
    α / l * sz * 1/2
end

function V(Ω, a, ad, sp, sm)
    Ω * (a * sp + ad * sm)
end

;